# Ftrace with cyclictest on Android

show how to use ftrace to get cyclictest trace data and analysis

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2017-12-07 07:11:31,464 INFO    : root         : Using LISA logging configuration:
2017-12-07 07:11:31,467 INFO    : root         :   /home/lisa/lisa-github/logging.conf


In [2]:
%pylab inline

import copy
import os
from time import sleep
from subprocess import Popen
import pandas as pd

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


## Test environment setup

For more details on this please check out **examples/utils/testenv_example.ipynb**.

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in `my_target_conf`. Run `adb devices` on your host to get the ID. Also, you have to specify the path to your android sdk in ANDROID_HOME.

In [3]:
# Setup a target configuration
my_target_conf = {
    
#     # Target platform and board
#     "platform"    : 'android',

#     # Add target support
#     "board" : 'pixel',
    
#     # Device ID
#     "device" : "HT6670300102",
    
#     "ANDROID_HOME" : "/home/vagrant/lisa/tools/android-sdk-linux/",
    
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : "/home/lisa/android-sdk",
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : '10.169.36.36',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
            "Device1" : 1, # iio:device0
        }
    },
    
    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "cpu_capacity",
            "cpu_frequency",
            "thermal_power_devfreq_limit",
            "thermal_power_devfreq_get_power",
            "thermal_power_cpu_limit",
            "thermal_power_cpu_get_power",
            "thermal_zone_trip",
            "cdev_update",
            "thermal_temperature",
            "thermal_power_allocator_pid",
            "thermal_power_allocator",
         ],
         "buffsize" : 100 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'taskset'],
    
    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],
}

In [4]:
my_tests_conf = {

    # Folder where all the results will be collected
    "results_dir" : "Ftrace_cyclictest",

    # Platform configurations to test
    "confs" : [
        {
            "tag"            : "antutu",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "ENERGY_AWARE",     # enable EAS
        },
    ],
}

In [5]:
# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

2017-12-07 07:11:34,633 INFO    : TestEnv      : Using base path: /home/lisa/lisa-github
2017-12-07 07:11:34,636 INFO    : TestEnv      : Loading custom (inline) target configuration
2017-12-07 07:11:34,642 INFO    : TestEnv      : Loading custom (inline) test configuration
2017-12-07 07:11:34,644 WARNING : TestEnv      : Wipe previous contents of the results folder:
2017-12-07 07:11:34,645 WARNING : TestEnv      :    /home/lisa/lisa-github/results/Ftrace_cyclictest
2017-12-07 07:11:34,647 INFO    : TestEnv      : External tools using:
2017-12-07 07:11:34,648 INFO    : TestEnv      :    ANDROID_HOME: /home/lisa/android-sdk
2017-12-07 07:11:34,650 INFO    : TestEnv      :    CATAPULT_HOME: /home/lisa/lisa-github/tools/catapult
2017-12-07 07:11:34,651 INFO    : TestEnv      : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
2017-12-07 07:11:34,653 INFO    : TestEnv      : Connecting Android target [0123456789ABCDEF]
2017-12-07 07:11:34,654 INFO    : TestEnv      : Connection settings

## Support Functions

This set of support functions will help us running the benchmark using different CPUFreq governors.

In [8]:
#Check if Benchmark si available on the device
target.install_if_needed(os.path.join(te.LISA_HOME, 'tools/wa_user_directory/plugins/cyclictest/bin/arm64/cyclictest'))

target.adb_root(force=True)

## Run test and collect trace data

After running the benchmark for the specified governors we can show and plot the scores:

In [31]:
cyclictest_path = "/data/local/tmp/bin/cyclictest"
tracecmd_path= "/data/local/tmp/bin/trace-cmd"

target.execute('/data/local/tmp/bin/trace-cmd reset')
target.execute('/data/local/tmp/bin/trace-cmd start -b 4000 -e irq -e sched -e timer -e workqueue -e cpu_idle -e thermal')
target.execute('/data/local/tmp/bin/cyclictest -t 8 -q -D 20')
target.execute('/data/local/tmp/bin/trace-cmd stop')
target.execute('/data/local/tmp/bin/trace-cmd extract -o /data/local/tmp/ftrace.dat')


'Kernel buffer statistics:\n  Note: "entries" are the entries left in the kernel ring buffer and are not\n        recorded in the trace data. They should all be zero.\n\nCPU: 0\nentries: 0\noverrun: 1352343\ncommit overrun: 0\nbytes: 1336\noldest event ts:  6718.236346\nnow ts:  6718.470559\ndropped events: 0\nread events: 72665\n\nCPU: 1\nentries: 0\noverrun: 1245127\ncommit overrun: 0\nbytes: 4036\noldest event ts:  6718.272383\nnow ts:  6718.470600\ndropped events: 0\nread events: 72297\n\nCPU: 2\nentries: 0\noverrun: 914357\ncommit overrun: 0\nbytes: 108\noldest event ts:  6718.216694\nnow ts:  6718.470637\ndropped events: 0\nread events: 72353\n\nCPU: 3\nentries: 0\noverrun: 583120\ncommit overrun: 0\nbytes: 2108\noldest event ts:  6718.120989\nnow ts:  6718.470672\ndropped events: 0\nread events: 72261\n\nCPU: 4\nentries: 0\noverrun: 0\ncommit overrun: 0\nbytes: 2548\noldest event ts:  6717.964318\nnow ts:  6718.470706\ndropped events: 0\nread events: 2764\n\nCPU: 5\nentries: 0\n

In [32]:
from devlib.utils.android import adb_command

adb_command(target.adb_name, 'pull /data/local/tmp/ftrace.dat ftrace.dat')

'[  0%] /data/local/tmp/ftrace.dat\n[  0%] /data/local/tmp/ftrace.dat\n[  0%] /data/local/tmp/ftrace.dat\n[  1%] /data/local/tmp/ftrace.dat\n[  1%] /data/local/tmp/ftrace.dat\n[  1%] /data/local/tmp/ftrace.dat\n[  2%] /data/local/tmp/ftrace.dat\n[  2%] /data/local/tmp/ftrace.dat\n[  2%] /data/local/tmp/ftrace.dat\n[  3%] /data/local/tmp/ftrace.dat\n[  3%] /data/local/tmp/ftrace.dat\n[  3%] /data/local/tmp/ftrace.dat\n[  4%] /data/local/tmp/ftrace.dat\n[  4%] /data/local/tmp/ftrace.dat\n[  4%] /data/local/tmp/ftrace.dat\n[  5%] /data/local/tmp/ftrace.dat\n[  5%] /data/local/tmp/ftrace.dat\n[  5%] /data/local/tmp/ftrace.dat\n[  5%] /data/local/tmp/ftrace.dat\n[  6%] /data/local/tmp/ftrace.dat\n[  6%] /data/local/tmp/ftrace.dat\n[  6%] /data/local/tmp/ftrace.dat\n[  7%] /data/local/tmp/ftrace.dat\n[  7%] /data/local/tmp/ftrace.dat\n[  7%] /data/local/tmp/ftrace.dat\n[  8%] /data/local/tmp/ftrace.dat\n[  8%] /data/local/tmp/ftrace.dat\n[  8%] /data/local/tmp/ftrace.dat\n[  9%] /data/local/

In [33]:
!ls -l ftrace.dat
!/usr/local/bin/trace-cmd report  ftrace.dat > ftrace.txt

-rw-r--r-- 1 lisa lisa 20758528 Dec  7 07:17 ftrace.dat
trace-cmd: No such file or directory
  [vmscan:mm_vmscan_writepage] function sizeof not defined
  [ras:mc_event] function mc_event_error_type not defined
  [libata:ata_qc_complete_internal] function libata_trace_parse_qc_flags not defined
  [libata:ata_qc_complete_failed] function libata_trace_parse_qc_flags not defined
  [libata:ata_qc_complete_done] function libata_trace_parse_qc_flags not defined
  [libata:ata_eh_link_autopsy_qc] function libata_trace_parse_qc_flags not defined
  [libata:ata_eh_link_autopsy] function libata_trace_parse_eh_action not defined
  [kvm:kvm_arm_set_regset] function sizeof not defined
  Error: expected type 5 but read 0
trace-cmd: Invalid argument
  [kmem:mm_page_pcpu_drain] function sizeof not defined
  [kmem:mm_page_free_batched] function sizeof not defined
  [kmem:mm_page_free] function sizeof not defined
  [kmem:mm_page_alloc_zone_locked] function sizeof not defined
  [kmem:mm_page_alloc_extfrag] 

In [34]:
!ls -l ftrace*

-rw-r--r-- 1 lisa lisa 20758528 Dec  7 07:17 ftrace.dat
-rw-rw-r-- 1 lisa lisa 41944355 Dec  7 07:17 ftrace.txt


In [35]:
trappy.plotter.plot_trace('ftrace.txt')